In [1]:
import os
import subprocess
import shlex
import pandas as pd
import numpy as np
import json
import os

In [ ]:
twitch_downloader_directory = r"C:\Users\trick\Desktop\twitchdownloader"

In [8]:
Symfuhny = {'url':'https://www.twitch.tv/videos/1797071998', 'name':'Symfuhny', 'beginning':'900', 'end':None, 'avg_viewers':5024, 'out_width':365, 'out_height':250, 'x':1298, 'y':241}
shroud = {'url':'https://www.twitch.tv/videos/1792790259', 'name':'shroud', 'beginning':'900', 'end':None, 'avg_viewers':11308, 'out_width':418, 'out_height':230, 'x':9, 'y':347}
cyr = {'url':'https://www.twitch.tv/videos/1797143072', 'name':'cyr', 'beginning':'900', 'end':None, 'avg_viewers':3437, 'out_width':396, 'out_height':367, 'x':1523, 'y':217}
NICKMERCS = {'url':'https://www.twitch.tv/videos/1798038394', 'name':'NICKMERCS', 'beginning':'3600', 'end':None, 'avg_viewers':14519, 'out_width':417, 'out_height':228, 'x':19, 'y':444}
Foolish_Gamers = {'url':'https://www.twitch.tv/videos/1798166416', 'name':'Foolish_Gamers', 'beginning':'900', 'end':None, 'avg_viewers':7856
, 'out_width':488, 'out_height':359, 'x':0, 'y':0}
Philza = {'url':'https://www.twitch.tv/videos/1798022118', 'name':'Philza', 'beginning':'900', 'end':None, 'avg_viewers':8163, 'out_width':554, 'out_height':330, 'x':0, 'y':398}
Xaryu = {'url':'https://www.twitch.tv/videos/1797143876', 'name':'Xaryu', 'beginning':'5400', 'end':None, 'avg_viewers':3473, 'out_width':518, 'out_height':373, 'x':11, 'y':561}
Ray = {'url':'https://www.twitch.tv/videos/1797237790', 'name':'Ray', 'beginning':'7200', 'end':None, 'avg_viewers':2134, 'out_width':433, 'out_height':203, 'x':12, 'y':71}
itsRyanHiga = {'url':'https://www.twitch.tv/videos/1799649695', 'name':'itsRyanHiga', 'beginning':'3600', 'end':None, 'avg_viewers':1656, 'out_width':357, 'out_height':259, 'x':0, 'y':92}
buckefps = {'url':'https://www.twitch.tv/videos/1797227298', 'name':'buckefps', 'beginning':'900', 'end':None, 'avg_viewers':2087, 'out_width':399, 'out_height':298, 'x':0, 'y':312}
Kastaclysm = {'url':'https://www.twitch.tv/videos/1798172429', 'name':'Kastaclysm', 'beginning':'3600', 'end':None, 'avg_viewers':1057, 'out_width':394, 'out_height':343, 'x':1525, 'y':0}

streamers = [Symfuhny, shroud, cyr, NICKMERCS, Foolish_Gamers, Philza, Xaryu, Ray, itsRyanHiga, buckefps, Kastaclysm]

In [ ]:
# DIRECTORTY SETUP
os.chdir(twitch_downloader_directory)

In [10]:
def FetchData(streamer):
    # DOWNLOAD CHAT LOGS
    command_line = f"TwitchDownloaderCLI.exe chatdownload --id {streamer['url']} -o {streamer['name']}-chat.json -b {streamer['beginning']}"
    if streamer['end'] is not None:
        command_line += f" -e {streamer['end']}"
    args = shlex.split(command_line)
    p = subprocess.Popen(args)
    p.wait()

    # GET DATA FROM CHAT LOGS
    data_json = json.load(open(f".../../../twitchdownloader/{streamer['name']}-chat.json", encoding="utf8"))
    streamer_name = data_json["streamer"]["name"]
    streamer_id = data_json["streamer"]["id"]
    end = (data_json["video"]["end"])
    end = end - ((end - int(streamer['beginning'])) % 15)
    comments = pd.DataFrame(data_json["comments"])
    data = comments["content_offset_seconds"].to_numpy()
    data = data[data < end]
    del comments

    # CALCULATE INTERACTIVITY
    interactivity = np.zeros(int((end - int(beginning)) / 15))
    for i in data:
        interactivity[(i - int(beginning)) // 15] += 1
    selected_indices = np.rint(np.arange(0,60) ** np.emath.logn(59,interactivity.shape[0]/2)).astype(int)
    interactivity_sorted = np.argsort(interactivity)[::-1][selected_indices]
    interactivity = interactivity / streamer['avg_viewers']

    # CREATE DATAFRAME, DOWNLOAD VIDEOS, CROP VIDEOS, AND EXTRACT AUDIO
    df = pd.DataFrame(columns=["streamer_name","streamer_id","start_seconds", "interactivity", "audio_path","video_path"])
    for i,j in enumerate(interactivity_sorted):
        command_line = f"TwitchDownloaderCLI.exe videodownload --id {streamer['url']} -o {streamer_name}-{i}.mp4 -b {j * 15 + int(streamer['beginning']) - 3} -e {(j+1) * 15 + int(streamer['beginning']) - 3}"
        args = shlex.split(command_line)
        p = subprocess.Popen(args)
        p.wait()
        command_line = f"ffmpeg -i {streamer_name}-{i}.mp4 -filter:v 'crop={streamer['out_width']}:{streamer['out_height']}:{streamer['x']}:{streamer['y']}' {streamer_name}-{i}-crop.mp4"
        args = shlex.split(command_line)
        p = subprocess.Popen(args)
        p.wait()
        command_line = f"ffmpeg -i {streamer_name}-{i}.mp4 {streamer_name}-{i}.mp3"
        args = shlex.split(command_line)
        p = subprocess.Popen(args)
        df = pd.concat([df,pd.DataFrame([{"streamer_name": streamer_name, "streamer_id": streamer_id, "start_seconds": j*15 + int(streamer['beginning']) - 3, "interactivity": interactivity[j],"audio_path":f"{streamer_name}-{i}.mp3","video_path":f"{streamer_name}-{i}.mp4"}])])
    df = df.reset_index().drop(columns=["index"])
    df.to_csv(f"{streamer_name}.csv")

In [11]:
# MOVE FILES TO OTHER FOLDER, MANUALLY DELETE VIDEOS THAT HAVE WRONG FACE CROPPING, THEN RUN THIS
def RenameFiles(streamer_name):
    os.chdir(f"C:\\Users\\trick\\Desktop\\twitchdownloader\\{streamer_name}")
    for i in range(60):
        if os.path.isfile(f"{streamer_name}-{i}-crop.mp4"):
            os.rename(f"{streamer_name}-{i}-crop.mp4", f"{streamer_name}-{i}.mp4")

In [ ]:
for streamer in streamers:
    FetchData(streamer)

In [ ]:
for streamer in streamers:
    RenameFiles(streamer['name'])

In [31]:
# Concatenate data
df = pd.DataFrame(columns=["streamer_name","streamer_id","start_seconds", "interactivity", "audio_path","video_path"])
for streamer in streamers:
    temp_df = pd.read_csv(f"{streamer['name']}/{streamer['name']}.csv", index_col=0)
    temp_df['audio_path'] = temp_df['audio_path'].apply(lambda x: f"data/{streamer['name']}/{x}")
    temp_df['video_path'] = temp_df['video_path'].apply(lambda x: f"data/{streamer['name']}/{x}")
    df = pd.concat([df,temp_df])
df = df.reset_index().drop(columns=["index"])

In [26]:
# normalize interactivity
df['interactivity'] = df['interactivity'] / df['interactivity'].max()

In [30]:
df.to_csv('data.csv')